In [ ]:
# -------------------------
# 통합 학습 + 테스트 + Optuna 최적화 + 시각화 모듈
# -------------------------
import os, sys, random, numpy as np, pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from pathlib import Path
from datetime import datetime
import optuna

# -------------------------
# 한글 폰트 설정
# -------------------------
def setup_font():
    font_path = "C:/Windows/Fonts/malgun.ttf"
    if os.path.exists(font_path):
        font_name = font_manager.FontProperties(fname=font_path).get_name()
        rc('font', family=font_name)
    else:
        rc('font', family='AppleGothic')  # MacOS 예시
    plt.rcParams['axes.unicode_minus'] = False

# -------------------------
# 재현성 설정
# -------------------------
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# -------------------------
# Seq2Seq CNN-LSTM 모델 정의
# -------------------------
class Seq2SeqCNNLSTM(nn.Module):
    def __init__(self, input_size=1, conv_channels=(32,16), lstm_hidden=64, output_steps=24, dropout=0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, conv_channels[0], 3, padding=1)
        self.bn1   = nn.BatchNorm1d(conv_channels[0])
        self.conv2 = nn.Conv1d(conv_channels[0], conv_channels[1], 3, padding=1)
        self.bn2   = nn.BatchNorm1d(conv_channels[1])
        self.relu  = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.encoder_lstm = nn.LSTM(conv_channels[1], lstm_hidden, batch_first=True)
        self.decoder_lstm = nn.LSTM(1, lstm_hidden, batch_first=True)
        self.fc = nn.Linear(lstm_hidden,1)
        self.output_steps = output_steps

    def forward(self, x, y=None, teacher_forcing_ratio=0.0):
        x = self.relu(self.bn1(self.conv1(x.transpose(1,2))))
        x = self.relu(self.bn2(self.conv2(x))).transpose(1,2)
        _, (hidden, cell) = self.encoder_lstm(x)
        decoder_input = x[:,-1,0].unsqueeze(-1)
        outputs = []
        for t in range(self.output_steps):
            decoder_output, (hidden, cell) = self.decoder_lstm(decoder_input.unsqueeze(1), (hidden,cell))
            out = self.fc(decoder_output).squeeze(1)
            outputs.append(out)
            decoder_input = y[:,t] if (y is not None and torch.rand(1).item() < teacher_forcing_ratio) else out
        return torch.stack(outputs, dim=1)

# -------------------------
# 시퀀스 생성
# -------------------------
def create_sequences(data, input_steps=168, output_steps=24):
    total_steps = len(data) - input_steps - output_steps + 1
    X = np.array([data[i:i+input_steps] for i in range(total_steps)])
    y = np.array([data[i+input_steps:i+input_steps+output_steps] for i in range(total_steps)])
    return X, y

# -------------------------
# Optuna objective
# -------------------------
def objective(trial, train_loader, val_loader, input_steps=168, output_steps=24, device="cpu"):
    conv1_ch = trial.suggest_int("conv1_ch", 16, 48, step=16)
    conv2_ch = trial.suggest_int("conv2_ch", 8, 32, step=8)
    lstm_hidden = trial.suggest_int("lstm_hidden", 32, 128, step=32)
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    model = Seq2SeqCNNLSTM(conv_channels=(conv1_ch, conv2_ch), lstm_hidden=lstm_hidden,
                           output_steps=output_steps, dropout=dropout).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scaler_amp = torch.amp.GradScaler()

    best_val = np.inf
    no_improve = 0
    epochs_trial = 10
    early_patience_trial = 3

    for epoch in range(epochs_trial):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device):
                y_pred = model(xb, yb, teacher_forcing_ratio=0.5)
                loss = criterion(y_pred, yb)
            scaler_amp.scale(loss).backward()
            scaler_amp.step(optimizer)
            scaler_amp.update()

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                with torch.amp.autocast(device_type=device):
                    val_loss += criterion(model(xb), yb).item()*xb.size(0)
        avg_val = val_loss / len(val_loader.dataset)

        if avg_val < best_val - 1e-6:
            best_val = avg_val
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= early_patience_trial:
                break
    return best_val

# -------------------------
# 전체 파이프라인
# -------------------------
def run_pipeline(train_csv, test_csv, target_col='합산발전량(MWh)', input_steps=168, output_steps=24,
                 batch_size=128, epochs=120, n_trials=10, lr=1e-3, output_dir='output'):

    setup_font()
    set_seed()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"💻 사용 디바이스: {device}")

    OUTPUT_DIR = Path(output_dir) / datetime.now().strftime("%Y%m%d_%H%M%S")
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

    train_df = pd.read_csv(train_csv, index_col=0, parse_dates=True)
    test_df  = pd.read_csv(test_csv, index_col=0, parse_dates=True)

    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_df[[target_col]].values)
    test_scaled  = scaler.transform(test_df[[target_col]].values)

    X_all, y_all = create_sequences(train_scaled, input_steps, output_steps)
    X_tensor = torch.tensor(X_all, dtype=torch.float32)
    y_tensor = torch.tensor(y_all, dtype=torch.float32)

    val_ratio = 0.1
    val_size = int(len(X_tensor)*val_ratio)
    train_size = len(X_tensor)-val_size
    train_dataset, val_dataset = random_split(TensorDataset(X_tensor, y_tensor), [train_size,val_size])

    num_workers = 0 if os.name=='nt' else 4
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    # -------------------------
    # Optuna 최적화
    # -------------------------
    study = optuna.create_study(direction="minimize")
    study.optimize(lambda trial: objective(trial, train_loader, val_loader, input_steps, output_steps, device),
                   n_trials=n_trials, show_progress_bar=True)
    best_params = study.best_params
    print(f"🏆 최적 하이퍼파라미터: {best_params}")

    # -------------------------
    # 최종 모델 학습
    # -------------------------
    model = Seq2SeqCNNLSTM(conv_channels=(best_params["conv1_ch"], best_params["conv2_ch"]),
                           lstm_hidden=best_params["lstm_hidden"],
                           output_steps=output_steps,
                           dropout=best_params["dropout"]).to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=best_params["lr"])
    scaler_amp = torch.amp.GradScaler()

    best_val_loss = np.inf
    no_improve = 0
    early_patience = 15
    history = {"train_loss":[], "val_loss":[], "val_rmse":[]}
    checkpoint_path = OUTPUT_DIR / "best_model.pt"

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        teacher_ratio = max(0.3, 0.7-0.4*(epoch-1)/epochs)
        for xb,yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            with torch.amp.autocast(device_type=device):
                y_pred = model(xb,yb,teacher_forcing_ratio=teacher_ratio)
                loss = criterion(y_pred,yb)
            scaler_amp.scale(loss).backward()
            scaler_amp.step(optimizer)
            scaler_amp.update()
            train_loss += loss.item()*xb.size(0)
        avg_train = train_loss/len(train_loader.dataset)

        model.eval()
        val_loss=0
        val_preds,val_trues=[],[]
        with torch.no_grad():
            for xb,yb in val_loader:
                xb,yb = xb.to(device), yb.to(device)
                with torch.amp.autocast(device_type=device):
                    y_pred = model(xb)
                val_loss += criterion(y_pred,yb).item()*xb.size(0)
                val_preds.append(y_pred.cpu().numpy())
                val_trues.append(yb.cpu().numpy())
        avg_val = val_loss/len(val_loader.dataset)
        val_preds = np.concatenate(val_preds,axis=0).reshape(-1,1)
        val_trues = np.concatenate(val_trues,axis=0).reshape(-1,1)
        val_rmse = np.sqrt(mean_squared_error(val_trues,val_preds))

        history["train_loss"].append(avg_train)
        history["val_loss"].append(avg_val)
        history["val_rmse"].append(val_rmse)

        if avg_val < best_val_loss-1e-6:
            best_val_loss = avg_val
            torch.save(model.state_dict(), checkpoint_path)
            no_improve = 0
        else:
            no_improve += 1
            if no_improve >= early_patience:
                print(f"✅ Early stopping at epoch {epoch}")
                break

        print(f"Epoch {epoch}/{epochs} | Train Loss:{avg_train:.6f} | Val Loss:{avg_val:.6f} | Val RMSE:{val_rmse:.2f} | TF:{teacher_ratio:.2f}")

    # -------------------------
    # 테스트 예측 + 분석
    # -------------------------
    model.load_state_dict(torch.load(checkpoint_path))
    model.eval()
    rolling_input = train_scaled[-input_steps:].tolist()
    predicted_scaled = []

    with torch.no_grad():
        i=0
        while i<len(test_scaled):
            steps_remaining = len(test_scaled)-i
            steps_to_predict = min(output_steps,steps_remaining)
            X_input = torch.tensor(rolling_input[-input_steps:],dtype=torch.float32).unsqueeze(0).to(device)
            y_pred = model(X_input,teacher_forcing_ratio=0.0).cpu().numpy().flatten()
            for step in range(steps_to_predict):
                rolling_input.append([y_pred[step]])
                predicted_scaled.append(y_pred[step])
            i += steps_to_predict

    predicted_scaled = np.array(predicted_scaled).reshape(-1,1)
    predicted_generation = scaler.inverse_transform(predicted_scaled)
    y_true = test_df[[target_col]].values
    rmse = np.sqrt(mean_squared_error(y_true,predicted_generation))
    r2   = r2_score(y_true,predicted_generation)
    print(f"테스트셋 평가 결과: RMSE={rmse:.2f}, R²={r2:.4f}")

    # -------------------------
    # 시각화 + CSV 저장
    # -------------------------
    train_min, train_max = train_df[target_col].min(), train_df[target_col].max()
    test_actual = y_true.flatten()
    test_pred_flat = predicted_generation.flatten()
    out_of_range_mask = (test_actual<train_min) | (test_actual>train_max)
    in_range_mask = ~out_of_range_mask

    # 범위 안/밖 RMSE/R²
    y_true_in  = test_actual[in_range_mask]
    y_pred_in  = test_pred_flat[in_range_mask]
    rmse_in    = np.sqrt(mean_squared_error(y_true_in, y_pred_in))
    r2_in      = r2_score(y_true_in, y_pred_in)

    y_true_out = test_actual[out_of_range_mask]
    y_pred_out = test_pred_flat[out_of_range_mask]
    rmse_out   = np.sqrt(mean_squared_error(y_true_out,y_pred_out)) if len(y_true_out)>0 else np.nan
    r2_out     = r2_score(y_true_out,y_pred_out) if len(y_true_out)>0 else np.nan
    out_ratio  = len(y_true_out)/len(test_actual)*100
    print(f"학습 범위 안: RMSE={rmse_in:.2f}, R²={r2_in:.4f}, 범위 밖 비율={out_ratio:.2f}%")

    # CSV 저장
    result_path = OUTPUT_DIR / "predicted_generation.csv"
    pd.DataFrame({
        "날짜": test_df.index,
        "실제발전량(MWh)": test_actual,
        "예측발전량(MWh)": test_pred_flat,
        "오차(MWh)": test_pred_flat-test_actual
    }).to_csv(result_path,index=False)
    print(f"✅ 테스트셋 예측 CSV 저장 완료: {result_path}")

    if len(y_true_out)>0:
        out_csv_path = OUTPUT_DIR / "out_of_range_generation.csv"
        pd.DataFrame({
            "날짜": test_df.index[out_of_range_mask],
            "실제발전량(MWh)": y_true_out,
            "예측발전량(MWh)": y_pred_out,
            "오차(MWh)": y_pred_out-y_true_out
        }).to_csv(out_csv_path,index=False)
        print(f"✅ 학습 범위 벗어난 값 CSV 저장 완료: {out_csv_path}")

    # 통합 시각화
    plt.figure(figsize=(16,7))
    plt.plot(test_df.index,test_actual,label="실제발전량",color="blue")
    plt.plot(test_df.index,test_pred_flat,label="예측발전량",color="red",alpha=0.7)
    plt.fill_between(test_df.index, train_min, train_max, color='yellow', alpha=0.2, label="학습 범위")
    if len(y_true_out)>0:
        plt.scatter(test_df.index[out_of_range_mask], y_true_out, color='black', label="학습 범위 밖 값", zorder=5)
    plt.title(f"테스트셋 예측 | RMSE={rmse:.2f}, R²={r2:.4f}")
    plt.xlabel("날짜")
    plt.ylabel("발전량 (MWh)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    graph_path = OUTPUT_DIR / "testset_analysis.png"
    plt.savefig(graph_path,dpi=300)
    plt.show()
    print(f"✅ 통합 분석 그래프 저장 완료: {graph_path}")

    return model, history, predicted_generation, OUTPUT_DIR